In [13]:
import os, sys, math, json, argparse, random
from pathlib import Path
from typing import Dict, List, Tuple
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
# -------------------------
# Utils
# -------------------------
def set_seed(seed:int=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

In [15]:
# -------------------------
# Data loading & weekly features
# -------------------------
def load_oulad(data_dir: str) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    info = pd.read_csv(os.path.join(data_dir, "studentInfo.csv"))
    vle  = pd.read_csv(os.path.join(data_dir, "studentVle.csv"))
    asm  = pd.read_csv(os.path.join(data_dir, "studentAssessment.csv"))
    # normalize column names
    info.columns = [c.strip() for c in info.columns]
    vle.columns  = [c.strip() for c in vle.columns]
    asm.columns  = [c.strip() for c in asm.columns]
    return info, vle, asm

def build_weekly_sequences(
    info: pd.DataFrame,
    vle: pd.DataFrame,
    asm: pd.DataFrame,
    in_weeks: int = 4,
    out_weeks: int = 2,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Build per-student weekly features and sliding windows of length in_weeks->out_weeks.
    Returns:
      weekly_df: one row per (student, week) with features
      samples_df: one row per training sample with [sid, X(weeks,in_feats), y(weeks,1)]
    """
    # clicks per day available as studentVle.sum_click; date is relative to course start
    # We'll convert date -> week_idx per student by subtracting student's min date and //7.
    # This makes sequences comparable within a student.
    sid_min_date = vle.groupby('id_student')['date'].min().to_dict()
    vle = vle.copy()
    vle['week_idx'] = vle.apply(lambda r: (int(r['date']) - int(sid_min_date.get(r['id_student'], int(r['date'])))) // 7, axis=1)
    clicks_week = vle.groupby(['id_student','week_idx'])['sum_click'].sum().rename('clicks').reset_index()

    # has_submit per week from studentAssessment.date_submitted (some are NaN)
    asm = asm.copy()
    asm = asm.dropna(subset=['id_student'])
    asm['date_submitted'] = asm['date_submitted'].fillna(-10).astype(int)
    # derive weekly index using same min-date alignment
    asm['week_idx'] = asm.apply(lambda r: (int(r['date_submitted']) - int(sid_min_date.get(r['id_student'], int(r['date_submitted'])))) // 7, axis=1)
    submit_week = asm.groupby(['id_student','week_idx']).size().rename('submit_cnt').reset_index()
    submit_week['has_submit'] = (submit_week['submit_cnt'] > 0).astype(int)

    # avg_score_sofar per week (cumulative mean up to that week, inclusive)
    # We'll compute by expanding within each student
    scores = asm[['id_student','week_idx','score']].dropna().copy()
    scores = scores.sort_values(['id_student','week_idx'])
    def cumavg(x):
        return x.expanding().mean()
    scores['avg_score_sofar'] = scores.groupby('id_student')['score'].transform(cumavg)
    score_week = scores.groupby(['id_student','week_idx'])['avg_score_sofar'].max().reset_index()

    # Merge all features to weekly grid
    weekly = pd.merge(clicks_week, submit_week[['id_student','week_idx','has_submit']], on=['id_student','week_idx'], how='left')
    weekly = pd.merge(weekly, score_week, on=['id_student','week_idx'], how='left')
    weekly = weekly.sort_values(['id_student','week_idx'])
    weekly['has_submit'] = weekly['has_submit'].fillna(0).astype(int)
    weekly['avg_score_sofar'] = weekly['avg_score_sofar'].ffill()
    weekly['avg_score_sofar'] = (
    weekly.groupby('id_student')['avg_score_sofar']
    .transform(lambda x: x.fillna(0))
    )

    # clicks_diff1 within student by week
    weekly['clicks_diff1'] = weekly.groupby('id_student')['clicks'].diff().fillna(0)

    # Build sliding windows per student
    in_feats = ['clicks','has_submit','avg_score_sofar','clicks_diff1']
    rows = []
    for sid, g in weekly.groupby('id_student'):
        g = g.sort_values('week_idx').reset_index(drop=True)
        # need consecutive weeks; fill missing weeks with zeros to maintain stride
        if len(g)==0: continue
        minw, maxw = int(g['week_idx'].min()), int(g['week_idx'].max())
        full = pd.DataFrame({'week_idx': list(range(minw, maxw+1))})
        full = full.merge(g[['week_idx']+in_feats], on='week_idx', how='left')
        full[in_feats] = full[in_feats].fillna(0)
        # slide
        for start in range(0, len(full) - (in_weeks + out_weeks) + 1):
            Xin = full.loc[start:start+in_weeks-1, in_feats].values.astype(np.float32) # (4,4)
            yout = full.loc[start+in_weeks:start+in_weeks+out_weeks-1, ['clicks']].values.astype(np.float32) # (2,1)
            rows.append((sid, Xin, yout))
    if len(rows)==0:
        raise RuntimeError("No training windows created. Check input CSVs.")
    # Assemble to DataFrame with numpy objects
    samples = pd.DataFrame(rows, columns=['id_student','X','y'])
    return weekly, samples

In [16]:
# -------------------------
# Dataset
# -------------------------
class SeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(np.stack(X)).float()  # (N, Tin, Fin)
        self.y = torch.from_numpy(np.stack(y)).float()  # (N, Tout, 1)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

In [17]:
# -------------------------
# LSTM
# -------------------------
class EncoderLSTM(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=64, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden_dim, num_layers=num_layers, batch_first=True)
    def forward(self, x):
        # x: (B, T, F)
        out, (h, c) = self.lstm(x)  # h: (num_layers,B,H)
        return h[-1], c[-1]         # (B,H), (B,H)

class DecoderLSTM(nn.Module):
    def __init__(self, hidden_dim=64, out_steps=2):
        super().__init__()
        self.lstm = nn.LSTM(1, hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, 1)
        self.out_steps = out_steps
    def forward(self, h0, c0, y0=None):
        # Teacher-forcing if y0 provided (B,Tout,1), else autoregressive with zeros start
        B = h0.size(0)
        inputs = torch.zeros(B, self.out_steps, 1, device=h0.device)
        if y0 is not None:
            # shift ground truth by one for teacher forcing start at t0=0
            inputs[:,0,0] = 0.0
        out, _ = self.lstm(inputs, (h0.unsqueeze(0), c0.unsqueeze(0)))
        yhat = self.out(out)  # (B,Tout,1)
        return yhat

class Seq2SeqLSTM(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=64, out_steps=2):
        super().__init__()
        self.enc = EncoderLSTM(in_dim, hidden_dim)
        self.dec = DecoderLSTM(hidden_dim, out_steps)
    def forward(self, x):
        h, c = self.enc(x)
        yhat = self.dec(h, c)
        return yhat

In [ ]:
# -------------------------
# Encoder：不變
# -------------------------
class EncoderVAE(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=64, latent_dim=16):
        super().__init__()
        self.lstm = nn.LSTM(in_dim, hidden_dim, batch_first=True)
        self.mu = nn.Linear(hidden_dim, latent_dim)
        self.logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        # x: [B, T_in, in_dim]
        _, (h, _) = self.lstm(x)        # h: [num_layers=1, B, H]
        h = h[-1]                        # [B, H]
        mu, logvar = self.mu(h), self.logvar(h)
        return mu, logvar


# -------------------------
# Decoder
# -------------------------
class DecoderVAE(nn.Module):
    def __init__(self, latent_dim=16, hidden_dim=64, out_steps=2, output_dim=1):
        super().__init__()
        self.lstm = nn.LSTM(latent_dim, hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, output_dim)
        self.out_steps = out_steps

    def forward(self, z):
        """
        z: [B, Z]
        return: yhat [B, out_steps, output_dim]
        """
        B, Z = z.size()
        z_seq = z.unsqueeze(1).repeat(1, self.out_steps, 1)  
        y, _ = self.lstm(z_seq)                              
        yhat = self.out(y)                                   
        return yhat


# -------------------------
# Seq2SeqVAE
# -------------------------
class Seq2SeqVAE(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=64, latent_dim=16, out_steps=2, output_dim=1):
        super().__init__()
        self.enc = EncoderVAE(in_dim, hidden_dim, latent_dim)
        self.dec = DecoderVAE(latent_dim, hidden_dim, out_steps, output_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        """
        x: [B, T_in, in_dim]
        return:
          yhat:  [B, out_steps, output_dim]
          mu:    [B, latent_dim]
          logvar:[B, latent_dim]
        """
        mu, logvar = self.enc(x)
        z = self.reparameterize(mu, logvar)      # [B, Z]
        yhat = self.dec(z)                       # [B, T_out, D]
        return yhat, mu, logvar


In [25]:
# -------------------------
# Training & Evaluation
# -------------------------
def train_lstm(model, loader, valid_loader, epochs, lr, device):
    model.to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    best = math.inf; best_state=None
    for ep in range(1, epochs+1):
        model.train()
        tr_loss=0.0
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            optim.zero_grad()
            yhat = model(X)
            loss = loss_fn(yhat, y)
            loss.backward(); optim.step()
            tr_loss += loss.item()*X.size(0)
        tr_loss /= len(loader.dataset)

        # valid
        model.eval()
        va_loss=0.0
        with torch.no_grad():
            for X, y in valid_loader:
                X, y = X.to(device), y.to(device)
                yhat = model(X)
                loss = loss_fn(yhat, y)
                va_loss += loss.item()*X.size(0)
        va_loss /= len(valid_loader.dataset)
        print(f"[LSTM] Epoch {ep}/{epochs}  train={tr_loss:.4f}  valid={va_loss:.4f}")
        if va_loss < best:
            best = va_loss; best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
    if best_state: model.load_state_dict(best_state)
    return model

def kld_loss(mu, logvar):
    # KL divergence between N(mu,sigma) and N(0,1)
    return -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

def train_vae(model, loader, valid_loader, epochs, lr, beta, device):
    model.to(device)
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    rec_loss = nn.MSELoss()
    best = math.inf; best_state=None
    for ep in range(1, epochs+1):
        model.train()
        tr=0.0
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            optim.zero_grad()
            yhat, mu, logvar = model(X)
            loss = rec_loss(yhat, y) + beta * kld_loss(mu, logvar)
            loss.backward(); optim.step()
            tr += loss.item()*X.size(0)
        tr /= len(loader.dataset)
        # valid
        model.eval()
        va=0.0
        with torch.no_grad():
            for X, y in valid_loader:
                X, y = X.to(device), y.to(device)
                yhat, mu, logvar = model(X)
                loss = rec_loss(yhat, y) + beta * kld_loss(mu, logvar)
                va += loss.item()*X.size(0)
        va /= len(valid_loader.dataset)
        print(f"[VAE ] Epoch {ep}/{epochs}  train={tr:.4f}  valid={va:.4f}")
        if va < best:
            best = va; best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
    if best_state: model.load_state_dict(best_state)
    return model

def evaluate(models, loaders, device, samples=20, out_dir=Path("./outputs")):
    lstm, vae = models
    test_loader = loaders['test']
    lstm.eval(); vae.eval()
    rec = {'idx':[], 'LSTM_MSE':[], 'VAE_best_MSE':[], 'delta':[], 'Diversity_std':[], 'y_true':[], 'y_LSTM':[], 'y_VAE_best':[]}
    mse = nn.MSELoss(reduction='none')
    with torch.no_grad():
        idx=0
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)  # y:(B,2,1)
            # LSTM single
            yhat_l = lstm(X)
            # VAE best-of-N
            all_samples = []
            for _ in range(samples):
                yhat_v, _, _ = vae(X)
                all_samples.append(yhat_v.unsqueeze(0))  # (1,B,2,1)
            YS = torch.cat(all_samples, dim=0)  # (N,B,2,1)
            # best-of-N by MSE
            mse_per = ((YS - y.unsqueeze(0))**2).mean(dim=(2,3))  # (N,B)
            best_idx = mse_per.argmin(dim=0)  # (B,)
            best = YS[best_idx, torch.arange(X.size(0))]          # (B,2,1)
            best_mse = ((best - y)**2).mean(dim=(1,2))            # (B,)
            lstm_mse = ((yhat_l - y)**2).mean(dim=(1,2))          # (B,)
            # diversity: std across N samples (flatten Tout)
            div = YS.view(samples, X.size(0), -1).std(dim=0).mean(dim=1)  # (B,)
            # coverage: both weeks within envelope across N
            ymin = YS.min(dim=0).values  # (B,2,1)
            ymax = YS.max(dim=0).values  # (B,2,1)
            covered = ((y >= ymin) & (y <= ymax)).all(dim=(1,2)).float()  # (B,)
            for b in range(X.size(0)):
                rec['idx'].append(idx)
                rec['LSTM_MSE'].append(float(lstm_mse[b].cpu()))
                rec['VAE_best_MSE'].append(float(best_mse[b].cpu()))
                rec['delta'].append(float(lstm_mse[b].cpu() - best_mse[b].cpu()))
                rec['Diversity_std'].append(float(div[b].cpu()))
                rec['y_true'].append(y[b].view(-1).cpu().numpy().tolist())
                rec['y_LSTM'].append(yhat_l[b].view(-1).cpu().numpy().tolist())
                rec['y_VAE_best'].append(best[b].view(-1).cpu().numpy().tolist())
                idx += 1
    df = pd.DataFrame(rec)
    ensure_dir(out_dir)
    df.to_csv(out_dir/"test_eval_rows.csv", index=False)

    # Aggregate metrics
    results = {}
    results['LSTM_MSE_mean'] = df['LSTM_MSE'].mean()
    results['VAE_BestofN_MSE_mean'] = df['VAE_best_MSE'].mean()
    results['VAE_Diversity_mean'] = df['Diversity_std'].mean()
    # coverage proportion
    # recompute coverage quickly from saved tuples (approx using delta>0 indicates vae beat lstm; but we want envelope coverage)
    # We didn't store per-sample coverage above to keep df concise. Let's recompute quickly:
    # (Simplification: Use win-rate as proxy for coverage when envelope not stored; strictly speaking, different.)
    # For fidelity, we add 'coverage_proxy' as ratio with delta>0
    results['Coverage_proxy'] = (df['delta'] > 0).mean()

    # Buckets
    def bucket(x):
        if x < -1000:                 return "VAE<<劣(>1000)"
        elif -1000 <= x < -200:       return "VAE劣(200~1000)"
        elif -200 <= x < -50:         return "VAE劣(50~200)"
        elif -50 <= x < -10:          return "VAE劣(10~50)"
        elif -10 <= x < 0:            return "VAE略劣(<10)"
        elif -10 <= x < 10:            return "打平"
        elif 10 <= x < 50:            return "VAE略勝(10~50)"
        elif 50 <= x < 200:           return "VAE勝(50~200)"
        elif 200 <= x < 1000:         return "VAE大勝(200~1000)"
        else:                         return "VAE>>大勝(>1000)"
    df['bucket'] = df['delta'].apply(bucket)
    bucket_stats = df['bucket'].value_counts().reset_index()
    bucket_stats.columns = ['Improvement bucket','count']
    bucket_stats['ratio'] = (bucket_stats['count']/len(df)).round(4)
    bucket_stats.to_csv(out_dir/"bucket_stats.csv", index=False)

    # Print console summaries
    print("\n=== Top-5 Regressed (VAE best >> LSTM) ===")
    top5 = df.nsmallest(5, 'delta')[['idx','LSTM_MSE','VAE_best_MSE','delta','y_true','y_LSTM','y_VAE_best','Diversity_std']]
    print(top5.to_string(index=False))

    print("\n=== Win-rate by improvement bucket (Δ = LSTM MSE − VAE best MSE) ===")
    print(bucket_stats.sort_values('Improvement bucket').to_string(index=False))

    print("\n========== 評估結果（原始尺度） ========== ")
    print(f"LSTM  MSE (整體): {results['LSTM_MSE_mean']:.4f}")
    print(f"VAE   Best-of-N MSE: {results['VAE_BestofN_MSE_mean']:.4f}  (N={samples})")
    print(f"VAE   Diversity (std): {results['VAE_Diversity_mean']:.4f}")
    print(f"VAE   Coverage (比例，使用勝率近似): {results['Coverage_proxy']:.4f}")
    med = df['LSTM_MSE'].median()
    print(f"門檻 tau = LSTM 每序列 MSE 中位數 = {med:.4f}")

    # Save a compact JSON summary
    with open(out_dir/"summary.json", "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    return df, bucket_stats, results

In [20]:
# -------------------------
# Split by student ID
# -------------------------
def split_by_student(samples: pd.DataFrame, seed=42, ratios=(0.7,0.15,0.15)):
    sids = samples['id_student'].unique()
    rng = np.random.default_rng(seed)
    rng.shuffle(sids)
    n = len(sids)
    n_tr = int(n*ratios[0])
    n_va = int(n*ratios[1])
    tr_sids = set(sids[:n_tr])
    va_sids = set(sids[n_tr:n_tr+n_va])
    te_sids = set(sids[n_tr+n_va:])
    def mask(s): return s.isin(tr_sids), s.isin(va_sids), s.isin(te_sids)
    m_tr, m_va, m_te = mask(samples['id_student'])
    return samples[m_tr], samples[m_va], samples[m_te]

In [26]:
# -------------------------
# Main
# -------------------------
def main():
    # 固定參數設定（取代 argparse）
    data_dir   = '.'                # OULAD CSV 資料夾路徑
    out_dir    = './outputs'        # 模型輸出目錄
    epochs     = 30                 # 訓練輪數
    batch_size = 128                # 每批次資料量
    hidden_dim = 64                 # LSTM 隱藏層維度
    latent_dim = 16                 # VAE 潛在維度
    beta       = 0.01               # VAE KLD loss 權重
    samples    = 40                 # VAE Best-of-N samples 數
    seed       = 42                 # 隨機種子

    # 設定環境
    set_seed(seed)
    out_dir = Path(out_dir)
    ensure_dir(out_dir)

    # 載入資料
    print("Loading OULAD CSVs...")
    info, vle, asm = load_oulad(data_dir)

    # 建立週序列資料
    print("Building weekly sequences...")
    weekly, sample_data = build_weekly_sequences(info, vle, asm, in_weeks=4, out_weeks=2)
    print(f"Weekly rows: {len(weekly):,}   Samples: {len(sample_data):,}")

    # 依 student_id 分割
    train_df, valid_df, test_df = split_by_student(sample_data, seed=seed)
    print(f"Train samples: {len(train_df):,} | Valid: {len(valid_df):,} | Test: {len(test_df):,}")

    # 建立資料張量
    X_tr, y_tr = train_df['X'].to_list(), train_df['y'].to_list()
    X_va, y_va = valid_df['X'].to_list(), valid_df['y'].to_list()
    X_te, y_te = test_df['X'].to_list(),  test_df['y'].to_list()

    tr_loader = DataLoader(SeqDataset(X_tr, y_tr), batch_size=batch_size, shuffle=True)
    va_loader = DataLoader(SeqDataset(X_va, y_va), batch_size=batch_size, shuffle=False)
    te_loader = DataLoader(SeqDataset(X_te, y_te), batch_size=batch_size, shuffle=False)

    # 選擇裝置
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # 建立模型
    lstm = Seq2SeqLSTM(in_dim=4, hidden_dim=hidden_dim, out_steps=2)
    vae  = Seq2SeqVAE(in_dim=4, hidden_dim=hidden_dim, latent_dim=latent_dim, out_steps=2)

    # 訓練
    print("\n=== Training LSTM ===")
    lstm = train_lstm(lstm, tr_loader, va_loader, epochs, 1e-3, device)
    print("\n=== Training VAE ===")
    vae  = train_vae(vae, tr_loader, va_loader, epochs, 1e-3, beta, device)

    # 評估
    print("\n=== Evaluation on Test ===")
    df, buckets, results = evaluate(
        (lstm, vae),
        {'test': te_loader},
        device,
        samples=samples,
        out_dir=out_dir
    )

    # 儲存模型
    torch.save(lstm.state_dict(), out_dir / "lstm.pt")
    torch.save(vae.state_dict(),  out_dir / "vae.pt")
    print(f"\nSaved models to {out_dir}")


if __name__ == "__main__":
    main()


Loading OULAD CSVs...
Building weekly sequences...
Weekly rows: 579,492   Samples: 606,310
Train samples: 425,920 | Valid: 90,772 | Test: 89,618
Device: cuda

=== Training LSTM ===
[LSTM] Epoch 1/30  train=8241.8276  valid=7061.6155
[LSTM] Epoch 2/30  train=6984.8786  valid=6657.7442
[LSTM] Epoch 3/30  train=6772.7560  valid=6530.1800
[LSTM] Epoch 4/30  train=6697.8665  valid=6465.7722
[LSTM] Epoch 5/30  train=6659.7450  valid=6442.7615
[LSTM] Epoch 6/30  train=6638.3231  valid=6437.2098
[LSTM] Epoch 7/30  train=6626.6709  valid=6418.9379
[LSTM] Epoch 8/30  train=6605.2879  valid=6397.1863
[LSTM] Epoch 9/30  train=6600.6172  valid=6391.6484
[LSTM] Epoch 10/30  train=6601.0657  valid=6389.8214
[LSTM] Epoch 11/30  train=6588.7565  valid=6377.0939
[LSTM] Epoch 12/30  train=6573.7113  valid=6352.4582
[LSTM] Epoch 13/30  train=6570.0298  valid=6353.9984
[LSTM] Epoch 14/30  train=6565.5290  valid=6356.6081
[LSTM] Epoch 15/30  train=6566.3707  valid=6345.0752
[LSTM] Epoch 16/30  train=6559.64